<a href="https://colab.research.google.com/github/wangjh789/2021-2_BigData/blob/master/ppjoin2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/gohdong/2021_autumn/master/BigData/project1/facebook_combined.txt 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz 
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install findspark
!pip install pyspark

--2021-10-07 15:53:57--  https://raw.githubusercontent.com/gohdong/2021_autumn/master/BigData/project1/facebook_combined.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 854362 (834K) [text/plain]
Saving to: ‘facebook_combined.txt.2’

facebook_combined.t 100%[===================>] 834.34K  --.-KB/s    in 0.05s   

2021-10-07 15:53:57 (17.3 MB/s) - ‘facebook_combined.txt.2’ saved [854362/854362]

--2021-10-07 15:53:59--  https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224445805 (214M) [application/x-gzip]
Saving to: ‘s

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext


sc = SparkContext("local","ppj")

In [3]:
undirect = sc.textFile('facebook_combined.txt').map(lambda x : x.split(" "))\
.flatMap(lambda x: ((int(x[0]),int(x[1])),(int(x[1]),int(x[0])))).groupByKey().mapValues(list)

threshold = 0.6

In [4]:
A =undirect.map(lambda x: len(x[1])).collect()
B=sorted(range(len(A)),key=lambda x:A[x])
order=sorted(range(len(A)),key=lambda x:B[x])
#sort를 위한 global order

In [5]:
sorted_undirect = undirect.mapValues(lambda x: sorted(x, key=lambda y : order[y]))

In [6]:
ref = sc.broadcast(dict(sorted_undirect.collect()))

In [7]:
def make_invert(item):
    key = item[0]
    arr= item[1]
    result= []
    prefix_len = int(len(ref.value[key]) - threshold*len(ref.value[key]) + 1) #prefix_filtering 
    for i in arr[:prefix_len]:
      result.append((i,key))
    return result

In [8]:
invert = sorted_undirect.flatMap(make_invert).groupByKey().mapValues(list)
invert.count()

2534

In [9]:
def make_pair(item):
    arr = item[1]
    result = []
    for idx,i in enumerate(arr):
        for j in arr[idx+1:]:
          if not len(ref.value[j]) >= threshold*len(ref.value[i]): continue #size filtering |x| >= t * |y| 
          result.append(((i,j),1))
    return result

In [10]:
pairs = invert.flatMap(make_pair).reduceByKey(lambda x,y : x+y) #prefix 부분에서의 overlap 계산
#invert != sorted_undirect
pairs.count()

403188

In [11]:
def compute_overlap(item):
    x,y = item[0][0],item[0][1]

    Px = int(len(ref.value[x]) - threshold*len(ref.value[x]) + 1) #prefix 길이
    Py = int(len(ref.value[y]) - threshold*len(ref.value[y]) + 1)

    Wx = ref.value[x][Px-1] # prefix의 마지막 토큰
    Wy = ref.value[y][Py-1]

    share = item[1] # prefix 부분의 overlap 수

    a = threshold/(1+threshold)*(len(ref.value[x])+len(ref.value[y]))
    value = 0
    
    if order[Wx] < order[Wy]:                           #x suffix 부분과 y prefix 부분이 겹칠 수 있는 경우
      value = share + len(set(ref.value[x][Px:])&set(ref.value[y][share:])) 

    else :                                              #y suffix 부분과 x prefix 부분이 겹칠 수 있는 경우
      value = share + len(set(ref.value[x][share:])&set(ref.value[y][Py:]))

    return value >= a

def is_not_friend(item):
    x = item[0][0]
    y = item[0][1]
    target,arr = x,ref.value[y]
    return not (target in arr)

In [12]:
result = pairs.filter(compute_overlap).filter(is_not_friend).flatMap(lambda x: ((x[0][0],x[0][1]),(x[0][1],x[0][0])))
result.count()

3238